# tweet_informer_provincia_ciudad

In [ ]:
from tweet_informer_lib import os, wget,shutil,pd,glob,sys,zipCompressDir,datetime
from tweet_informer_lib import make_cammesa_url,make_plt_provincia_capital_bars,cammesa_consume_reader
from tweet_informer_lib import json,tweepy,reduce

In [ ]:
pd_cfg=pd.read_csv('cfg/csv_cfg_provincias.csv',index_col=0)
prov_dict=pd_cfg.T.to_dict()

### url

In [ ]:
url_dict={}
for opt in prov_dict:
    url_dict.update({opt:{}})
    provincia,capital=prov_dict[opt]['Total'],prov_dict[opt]['Capital']
    make_cammesa_url('provincia',provincia,url_dict[opt])
    make_cammesa_url('capital',capital,url_dict[opt])

### make dirs

In [ ]:
tmp='tmp_multi'
arxive='arxive'
for folder in [arxive,tmp]:
    os.makedirs(folder,exist_ok=True)

### get data

In [ ]:
csv_dict={}
req_time=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
for p in url_dict:
    csv_dict.update({p:{}})
    for key in url_dict[p]:
        filename=os.path.join(tmp,p+'_'+key+'.csv')
        filesaved = wget.download(url_dict[p][key],out=filename)
        csv_dict[p].update({key:filesaved})

In [ ]:
csv_=glob.glob(os.path.join(tmp,'*.csv'))
if len(csv_)>0:
    pass
else:
    shutil.rmtree(tmp)
    sys.exit('No CSV files')

### data proc

In [ ]:
aux=[]
for p in csv_dict:
    for c in csv_dict[p]:
        df=cammesa_consume_reader(csv_dict[p][c],[1,2])
        df.columns=[p+'_'+c]
        aux.append(df)
df=reduce(lambda x, y: pd.merge(x, y, left_index=True,right_index=True), aux)
df[df < 0] = 0
df_notna=df.dropna()

In [ ]:
if len(df_notna)>1:
    pass
else:
    shutil.rmtree(tmp)
    sys.exit('Short Dataframe')

In [ ]:
dend=df_notna.index[-1].strftime('%d-%m-%Y %H:%M')
pd_in=pd.DataFrame(df_notna.iloc[-1].values.reshape(-1,2),index=csv_dict.keys(),columns=['Provincia','Capital'])
pd_in['Provincia_SIN_Capital']=pd_in['Provincia']-pd_in['Capital']

In [ ]:
try:
    fig=make_plt_provincia_capital_bars(pd_in,title='Demanda Real [MW] ' + dend)
    # save fig
    figName=os.path.join(tmp,'consumo.png')
    fig.savefig(figName,transparent=False)
except:
    shutil.rmtree(tmp)
    print('Plot ERROR')
    sys.exit('Plot ERROR')   

### tweeter Stuff

In [ ]:
#
config_file = 'cfg/.tweepy.json'
try:
    with open(config_file) as fh:
        config = json.load(fh)
        auth = tweepy.OAuthHandler(config['consumer_key'], config['consumer_secret'])
        auth.set_access_token(config['access_token'], config['access_token_secret'])
        twitter = tweepy.API(auth)
        tweet ='Consumo [MW] - Provincias'
        image_path =figName
        # to attach the media file 
        status = twitter.update_with_media(image_path, tweet)
except:
    shutil.rmtree(tmp)
    sys.exit('Failed to TWEET')

### Compress - Save - Delete

In [ ]:
zipname=os.path.join(arxive,req_time+'_consumo_Provincias.zip')
zipCompressDir(zipname,tmp)

In [ ]:
shutil.rmtree(tmp)